# BioConceptVec Tutorial

This tutorial provides a fundemental introduction to our BioConceptVec models. It illustrates (1) how to load the model, (2) how to get concept vectors, and (3) how to search top K similar concepts. [Source](https://github.com/ncbi/BioConceptVec/blob/master/bioconcept_tutorial.ipynb)

## 1. Prerequisites

Install gensim to load BioConceptVec.

In [1]:
from gensim.models import KeyedVectors
import os, sys, json, numpy as np

## 2. Load BioConceptVec

Go to data.ipynb to download the data. Then set the path to the data directory.

In [2]:
YOUR_BIOCONCEPTVEC_PATH = '../embeddings/bioconceptvec_glove.bin'
YOUR_JSON_PATH = '../embeddings/concept_glove.json'

Let's create a function using gensim to load BioConceptVec.

In [3]:
def load_embedding(path, binary):
    embedding = KeyedVectors.load_word2vec_format(path, binary=binary)
    print('embedding loaded from', path)
    return embedding

Let's load any one version of BioConceptVec. This might take a few minutes to load.

In [4]:
model = load_embedding(YOUR_BIOCONCEPTVEC_PATH, binary=True)

embedding loaded from ../embeddings/bioconceptvec_glove.bin


## 3. Load concept vectors only (an alternative approach)

If you only need concept vectors rather than other common word vectors, you could also load the json file that contains concept vectors only.

In [5]:
with open(YOUR_JSON_PATH) as json_file:  
    concept_vectors = json.load(json_file)
print('load', len(concept_vectors), 'concepts')

load 402712 concepts


In [6]:
import pandas as pd
keys = concept_vectors.keys()
# Turn keys into a dataframe
df = pd.DataFrame(list(keys)[:500], columns=['concept'])

## 4. Get concept vectors

Now you could specify a concept ID and get the vector.

For the complete BioConceptVec model, you could use:

In [6]:
concept_vec = model['Gene_2997']
concept_vec

array([-0.205002, -0.858642,  0.293454,  0.420697,  0.819198, -0.014607,
        0.648112,  0.375288,  0.402923, -0.486146,  0.061995, -0.597637,
       -0.18606 , -0.030133,  0.057708,  0.0928  , -0.231601, -0.102536,
        0.113617,  0.096256, -0.716546,  0.129387,  0.097503,  0.516359,
        0.150461, -0.128564,  0.617396, -0.70626 , -0.214449, -0.609342,
       -0.610461,  0.202288,  0.509137,  0.601489,  0.49775 ,  0.046024,
        0.477214,  0.170123,  0.8417  ,  0.35232 ,  0.084167,  0.006656,
        0.052083,  0.27484 , -0.222268,  0.738473, -0.021782,  0.917502,
        0.208649,  0.21989 , -0.262444,  0.110046,  0.838382,  0.243271,
        0.584187, -0.17198 , -0.35021 ,  0.255658,  0.495955, -0.438681,
       -0.480301, -0.998345, -0.303257,  0.60441 , -0.235623,  0.19677 ,
       -0.311972,  0.406385,  0.067994,  0.487211, -0.016605, -0.42778 ,
       -1.023635,  0.342992, -0.103875,  0.076305,  0.890983,  0.101807,
       -0.019141, -0.469659, -0.081776, -0.074963, 

For the json file, it is a dictionary, so the code is exactly the same:

In [7]:
np.array(concept_vectors['Gene_2997'])

array([-0.20500199, -0.85864198,  0.29345399,  0.420697  ,  0.81919801,
       -0.014607  ,  0.648112  ,  0.37528801,  0.40292299, -0.486146  ,
        0.061995  , -0.597637  , -0.18606   , -0.030133  ,  0.057708  ,
        0.0928    , -0.231601  , -0.102536  ,  0.113617  ,  0.096256  ,
       -0.716546  ,  0.12938701,  0.097503  ,  0.51635897,  0.150461  ,
       -0.128564  ,  0.617396  , -0.70626003, -0.214449  , -0.60934198,
       -0.610461  ,  0.202288  ,  0.50913697,  0.60148901,  0.49775001,
        0.046024  ,  0.47721401,  0.170123  ,  0.84170002,  0.35231999,
        0.084167  ,  0.006656  ,  0.052083  ,  0.27484   , -0.222268  ,
        0.738473  , -0.021782  ,  0.91750199,  0.20864899,  0.21989   ,
       -0.26244399,  0.110046  ,  0.83838201,  0.24327099,  0.58418697,
       -0.17197999, -0.35021001,  0.255658  ,  0.49595499, -0.43868101,
       -0.48030099, -0.99834502, -0.30325699,  0.60440999, -0.235623  ,
        0.19677   , -0.31197199,  0.406385  ,  0.067994  ,  0.48

Similarly, you could use the concept IDs that we provided in the json file to get more concept vectors.

In [7]:
import pandas as pd
df = pd.read_csv('../datasets/bioconcepts2pubtatorcentral')
df


ParserError: Error tokenizing data. C error: Expected 1 fields in line 79, saw 2


## 5.Compute the similarity between concepts

Now we can use the concept vectors to find similar concepts.

First let's create a function to calculate the Cosine similarity

In [8]:
def cosine(a, b):
    norm1 = np.linalg.norm(a)
    norm2 = np.linalg.norm(b)
    return np.dot(a, b) / (norm1 * norm2)
    

For example, for the interleukin 10 gene (Gene_3586), let's find out which gene is more similar to it, interleukin 4 gene (Gene_3565) or HUWE1 (Gene_10075)?

In [9]:
cosine(model['Gene_3586'], model['Gene_3565'])

0.88254434

In [10]:
cosine(model['Gene_3586'], model['Gene_10075'])

0.1340888

The results show that interleukin 4 gene is more similar to interleukin 10 gene. Indeed, they share GO terms.

## 6. Find top K similar terms via BioConceptVec

You can also use the embedding to find out the top K similar terms:

In [9]:
model.most_similar(positive=['Gene_432248'], topn=3)

[('Gene_399342', 0.6710575222969055),
 ('Gene_399275', 0.6615089774131775),
 ('Gene_378586', 0.6413488984107971)]

In [ ]:
chunksize = 10 ** 6  # adjust this value depending on your available memory
chunks = []
for chunk in pd.read_csv('file.csv', chunksize=chunksize):
    chunks.append(chunk)

df = pd.concat(chunks, axis=0)
del chunks  # free up memory

In [10]:
import dask.dataframe as dd

df = dd.read_csv('/Users/danielgeorge/Documents/work/ml/bioconceptvec-explorer/bioconceptvec-explorer/datasets/bioconcepts2pubtatorcentral')

In [11]:
with open("/Users/danielgeorge/Documents/work/ml/bioconceptvec-explorer/bioconceptvec-explorer/datasets/bioconcepts2pubtatorcentral") as infile:
    for line in infile:
        print(line)

ParserError: Error tokenizing data. C error: Expected 1 fields in line 79, saw 2
